In [69]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

In [87]:
# use pandas to store the data s a dataframe
df = pd.read_csv('datasets/credit.csv', header=0)

In [88]:
df.shape

(400, 12)

In [91]:
# TODO simulate default

In [89]:
df.columns

Index(['Unnamed: 0', 'Income', 'Limit', 'Rating', 'Cards', 'Age', 'Education',
       'Gender', 'Student', 'Married', 'Ethnicity', 'Balance'],
      dtype='object')

In [90]:
df.head()

,Unnamed: 0,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
0,1,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
1,2,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903
2,3,104.593,7075,514,4,71,11,Male,No,No,Asian,580
3,4,148.924,9504,681,3,36,11,Female,No,No,Asian,964
4,5,55.882,4897,357,2,68,16,Male,No,Yes,Caucasian,331


In [92]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Balance'], axis=1), 
                                                    df['Balance'],
                                                    test_size=0.2, random_state=42)

In [93]:
X_train.head()

,Unnamed: 0,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity
3,4,148.924,9504,681,3,36,11,Female,No,No,Asian
18,19,49.570,6384,448,1,28,9,Female,No,Yes,Asian
202,203,21.038,1448,145,2,58,13,Female,No,Yes,Caucasian
250,251,10.363,2430,191,2,47,18,Female,No,Yes,Asian
274,275,30.550,5869,439,5,81,9,Female,No,No,African American


In [94]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 3 to 102
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  320 non-null    int64  
 1   Income      320 non-null    float64
 2   Limit       320 non-null    int64  
 3   Rating      320 non-null    int64  
 4   Cards       320 non-null    int64  
 5   Age         320 non-null    int64  
 6   Education   320 non-null    int64  
 7   Gender      320 non-null    object 
 8   Student     320 non-null    object 
 9   Married     320 non-null    object 
 10  Ethnicity   320 non-null    object 
dtypes: float64(1), int64(6), object(4)
memory usage: 30.0+ KB


In [95]:
for i in X_train.columns:
    print(i)
    print(X_train[i].value_counts(dropna=False), "\n")

Unnamed: 0
2      1
3      1
277    1
276    1
275    1
      ..
140    1
139    1
138    1
137    1
400    1
Name: Unnamed: 0, Length: 320, dtype: int64 

Income
23.793     2
20.974     1
27.847     1
62.328     1
44.522     1
          ..
18.701     1
33.694     1
27.241     1
27.999     1
146.183    1
Name: Income, Length: 319, dtype: int64 

Limit
3388    2
5140    2
2860    2
4391    2
855     2
       ..
3976    1
5149    1
5096    1
3063    1
2047    1
Name: Limit, Length: 315, dtype: int64 

Rating
287    4
383    4
369    3
263    3
321    3
      ..
224    1
228    1
229    1
231    1
511    1
Name: Rating, Length: 249, dtype: int64 

Cards
2    95
3    90
4    50
1    42
5    28
6    11
7     2
8     1
9     1
Name: Cards, dtype: int64 

Age
44    12
66    11
67     9
71     9
50     9
      ..
89     1
23     1
85     1
27     1
98     1
Name: Age, Length: 64, dtype: int64 

Education
14    41
15    39
16    39
17    32
13    31
11    24
12    24
9     22
10    18
18    17


In [102]:
# TODO: change this into a string value in the dataset to have an example of encoding
X_train['Education'].unique()

array([11,  9, 13, 18, 17, 16,  7, 14, 19,  8, 15, 10, 20, 12,  6])

Column classifications

In [103]:
def order_education(X):
#     levels = {
#         1: 0
#     }
#     value = pd.DataFrame(X).applymap(lambda x: levels[x])
    return 

In [104]:
X_train.columns

Index(['Unnamed: 0', 'Income', 'Limit', 'Rating', 'Cards', 'Age', 'Education',
       'Gender', 'Student', 'Married', 'Ethnicity'],
      dtype='object')

In [111]:
continuous = ['Income', 'Limit', 'Rating', 'Cards', 'Age']

ordinal = ['Education']

categorical_with_missing = ['Gender', 'Student', 'Ethnicity', 'Married']

categorical_without_missing = []

The plan is to scale all continuous variables from [0, 1], establish one-hot encode all categories excep the missing values, and one-hot encode all of the other categorical values without a missing value

column transofrmations

In [112]:
# impute any missing values with the mean
cont_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scale', MinMaxScaler(feature_range=(0, 1)))
])

ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal_encoding', FunctionTransformer(func=order_education, validate=False))
])

cat_with_unknown_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values='?', strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers = [
        ('cont', cont_transformer, continuous),
        ('ordinal', ordinal_transformer, ordinal),
        ('cat_with_unknown', cat_with_unknown_transformer, categorical_with_missing),
        ('cat', cat_transformer, categorical_without_missing)
    ]
)

In [113]:
preprocessor.fit_transform(X_train, y_train)

AttributeError: 'NoneType' object has no attribute 'size'

Since this is a class imbalance problem, try two different grid searches over two classification algorithms that have class weights\
We'll use chi2 feature selector

In [ ]:
lr_pipe = Pipelien(steps=[('preprocessing', preprocessor),
                         ('feat_select', SelectKBest()),
                         ('lr_model', LogisticRegression(class_weight='balanced', solver='liblinear', 
                                                         random_state=seed_val)
                         )])

reg_terms = [1000/10**(2*i) for i in range(5)]

In [ ]:
param_grid = {
    'lr_model__C': reg_terms,
    'feat_select__k': [10, 20, 30, 'all'],
}

In [ ]:
lr_grid = GridSearchCV(lr_pipe, cv=7, scoring='roc_auc', param_grid=param_grid, return_train_score=False)

In [ ]:
lr_grid.fit(X_train, y_train)

In [ ]:
lr_results = pd.DataFrame(lr_grid.cv_results_)

In [ ]:
lr_results[["mean_test_score","std_test_score","param_rf_model__max_depth","param_rf_model__n_estimators"]].sort_values("mean_test_score", ascending=False)

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, n_jobs=1, train_sizes=np.linspace(0.1, 1.0, 10)):
    """
    Generate a simple plot of the test and training learning curve
    """
    
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training Examples")
    plt.ylabel("Score")
    
    train_sizes, train_scores, test_scores = learning_curve(estimator,
                                                           x,
                                                           y,
                                                           cv=cv,
                                                           n_jobs=n_jobs,
                                                           train_sizes=train_sizes,
                                                           scoring='roc_auc')
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
    axes[0].fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
    axes[0].plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
    axes[0].plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
    axes[0].legend(loc="best")
    
    return plt

best_lr_pipe = Pipeline(steps=[('preprocessing', preprocessor),
                              ('lr_model', LogisticRegression(class_weight='balanced', C=1000, solver='liblinear',
                                                             random_state=seed_val))])
plot_learning_curve(best_lr_pipe, 'Tuned LR Learning Curves', X_train, y_train, cv=7)
plt.show()

The model looks to have high bias

The final holdout evaluation is

In [ ]:
final_pipe = Pipeline(steps=[('preprocessing', preprocessor),
                            ('lr_model', LogisticRegression(class_weight='balanced', C=1000, solver='liblinear', 
                                                           random_state=seed_val))])

final_pipe.fit(X_train, y_train)
x_probas = final_pipe.predict_proba(x_test)

roc_auc_score(y_test, x_probas[:, 1])

In [ ]:
final_pipe.fit(df_training.drop(['race', 'target'], axis=1), df_training['targe'])